Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. Hyundai Heavy Industries is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise ships.
    Variables/Columns
    Ship Name 1-20
    Cruise Line 21-40
    Age (as of 2013) 46-48
    Tonnage (1000s of tons) 50-56
    passengers (100s) 58-64
    Length (100s of feet) 66-72
    Cabins (100s) 74-80
    Passenger Density 82-88
    Crew (100s) 90-96

Your job is to help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis!

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.functions import corr

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [ ]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [ ]:
# Load data
df = spark.read.csv("../../Data/cruise_ship_info.csv", header=False, inferSchema=True)

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df.describe().show()

In [ ]:
df.groupBy('Cruise_line').count().show()

In [ ]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.transform(df)
indexed.head(5)

In [ ]:
indexed.columns

In [ ]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'cruise_cat'], outputCol='features')

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.select('features', 'crew').show(10)

In [ ]:
final_data = output.select('features', 'crew')

In [ ]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [ ]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [ ]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [ ]:
# Print the coeffecients and intercept for linear regression
print('Coefficients: {} \nIntefercept {}'.format(lrModel.coefficients, lrModel.intercept))

In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
print('RMSE: {}'.format(test_results.rootMeanSquaredError))
print('MSE: {}'.format(test_results.meanSquaredError))
print('R2: {}'.format(test_results.r2))

In [ ]:
df.select(corr('crew', 'passengers')).show()

In [ ]:
df.select(corr('crew', 'cabins')).show()